In [1]:
import html5lib.treebuilders.base
html5lib.treebuilders._base= html5lib.treebuilders.base
from bs4 import BeautifulSoup

In [136]:
text_file = open('resources/scraped/1/1.html', 'r')
html = text_file.read()


In [142]:
s= BeautifulSoup(html,"lxml")

# If parent.parent.attrs is xg_module_body then it means it is the topmost question
# If parent.parent.attrs is i0,i1 etc. then we need to establish the ordering of 
# questions and answers. FOr more info run the following commented code.
for descriptions in s.select('div.description'):
    print(descriptions.parent.parent.attrs['class'])


# Find all <p> tags in div.description and extract their text. The concatenated result
# will form the body of the reply (question or answer). For more info run the following code.
for description in s.select('div.description'):
    #Get the content of the description in a variable 'res'
    res=""
    for pars in description.find_all('p'):
        res = res+'\n'+(pars.getText())
    #find the parent.parent.attrs of this description and set it as question/answer
    #accordingly
    if description.parent.parent.attrs['class'][0] == "xg_module_body":
        #means it is topmost question
        res='QUESTION: \n'+res
    elif description.parent.parent.attrs['class'][0] == "discussion":
        #means it is one of the discussion, now extract it's id and based on that 
        #set it as a reply to previous question of a new question.
        res='DISCUSSION:\n'+res
        #extract its number
        num = description.parent.parent.attrs['class'][2]
        res = res+'\n'+num
    else:
            print('Serious error: Not accounted for')
    print(res+'\n-------------------------------------------\n')




['xg_module_body']
['discussion', 'clear', 'i0', 'xg_lightborder']
['discussion', 'clear', 'i1', 'xg_lightborder']
['discussion', 'clear', 'i2', 'xg_lightborder']
['discussion', 'clear', 'i3', 'xg_lightborder']
['discussion', 'clear', 'i4', 'xg_lightborder']
QUESTION: 

Hi all,
Please suggest a few good schools in and near Indirapuram. My 
daughter is 3+ and i am interested in schools which do not overburden 
small kids with lot of academic work.
Could you please rate these schools on the above mentioned criteria :
DPS Indirapuram
Amity Vasundhara sec 1 & 6
Father Agnel, Noida
St. Francis, Vasundhara
Cambridge, Indirapuram

Thanks!!
-------------------------------------------

DISCUSSION:

Gargi
See this
http://www.admissionsnursery.com/forum/topics/nursery-admissions-in...
i0
-------------------------------------------

DISCUSSION:

thanks a lot Sakshi !
Please give some idea about what we need to keep handy before filling any online form.
Tnx in advance :)
i1
------------------------